In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

This notebook has been scripted for use in condor:
* `conjob.cmd` + `counts.py` + `counts.sh` for unsat counts on existing cnfs 
* `conjob.cmd` + `counts_fresh.py` + `counts_fresh.sh` for generating new cngs with unsat counts

In this notebook we show an example of generating random SAT instances.

In [2]:
from benchmark.cnf.random_cnf import RandomCNF
from benchmark.cnf.generator.ksat_generator import KSATGenerator
from benchmark.cnf.generator.knaesat_generator import KNAESATGenerator

# Random ksat from generator
ksg = KSATGenerator()
rp = RandomCNF(generator=ksg)
rks = rp.from_poisson(n=12, k=3, satisfiable=False)

In [9]:
# Random ksat 
rp = RandomCNF(type="ksat")
rks = rp.from_poisson(n=12, k=5, satisfiable=True)[0]

In [10]:
# Random knaesat
rp = RandomCNF(type='knaesat')
rkns = rp.from_poisson(n=12, k=3, satisfiable=True)[0]

Unsatisfiable formula generated, trying again
Unsatisfiable formula generated, trying again
Unsatisfiable formula generated, trying again


In [24]:
c = rkns.clauses[0]
print(c)
c.is_satisfied('111111101111')

(¬x7 ∨ x8 ∨ x10)


False